In [1]:
import json
import random
from collections import defaultdict
from openai import OpenAI
from tqdm import tqdm
import ast
import re
import time
import os
import pprint
from itertools import product, islice
import tiktoken
import json
import hashlib
import nest_asyncio
import asyncio
import aiohttp
import time
from tqdm.asyncio import tqdm as tqdm_async


In [2]:
with open("/Users/paolocadei/Documents/Masters/Thesis/Spider2/api_keys/api_keys.json", "r") as f:
    api_keys = dict(json.load(f))

client = OpenAI(
        api_key = api_keys["open_ai_key"],
    )
    
model = "gpt-4o-mini-2024-07-18"

In [3]:
with open("/Users/paolocadei/Documents/Masters/Thesis/Spider2/1_sampled_schema_groups_with_metadata.json", "r") as f:
    file = dict(json.load(f))

### Initialising the column descriptions

Here we format better the structure of the column descriptions and also convert all empty strings and NaN to None for standardisation.

In [4]:
for database in file.keys():

    for table in file[database].keys():

        for template in file[database][table]['grouped'].keys():

            for t in range(len(file[database][table]['grouped'][template])):

                column_names = file[database][table]['grouped'][template][t]['details']['columns'].keys()
                column_descriptions = file[database][table]['grouped'][template][t]['details']['description']

                file[database][table]['grouped'][template][t]['details']['description'] = {
                                        col_n: col_d
                                            if col_d and isinstance(col_d, str) and len(col_d.split()) > 5 else None
                                        for col_n, col_d in zip(column_names, column_descriptions)
                                    }

        for t in file[database][table]['ungrouped']:

            # restructuring file
            column_names_types = file[database][table]['ungrouped'][t]['columns']
            file[database][table]['ungrouped'][t].pop('columns', None)

            sample_row = file[database][table]['ungrouped'][t]['sample_row']
            file[database][table]['ungrouped'][t].pop('sample_row', None)

            with open(f'/Users/paolocadei/Documents/Masters/Thesis/Spider2/spider2-snow/resource/databases/{database}/{table}/{t}') as f:
                table_info = json.load(f)

            col_descriptions = table_info['description']

            file[database][table]['ungrouped'][t]['details'] = {
                'columns': column_names_types,
                'description': {
                                    col_n: col_d
                                        if col_d and isinstance(col_d, str) and len(col_d.split()) > 5 else None
                                    for col_n, col_d in zip(column_names_types.keys(), col_descriptions)
                                },
                'sample_row': sample_row
            }


### Inserting the already OpenAI generated descriptions

In [5]:
import json

for responses in ['2_successes.jsonl', '2_successes_multiple.jsonl']:

    # Load JSONL responses
    with open(responses, 'r') as f:
        success_lines = [json.loads(line) for line in f]

    # Build lookups
    ungrouped_lookup = {}
    grouped_lookup = {}

    for entry in success_lines:
        if entry['status'] != 'success':
            continue

        if entry['ungrouped_key'] is not None:
            ungrouped_lookup[(entry['database'], entry['table'], entry['ungrouped_key'], entry['column'])] = entry['response']['description']
        else:
            grouped_lookup[(entry['database'], entry['table'], entry['template'], entry['entry_index'], entry['column'])] = entry['response']['description']

    # Utility: count None descriptions
    def count_none_descriptions(file):
        count = 0
        for db in file:
            for tbl in file[db]:
                # Grouped
                for tmpl in file[db][tbl].get('grouped', {}):
                    for t in file[db][tbl]['grouped'][tmpl]:
                        count += sum(1 for desc in t['details']['description'].values() if desc is None)
                # Ungrouped
                for u_key in file[db][tbl].get('ungrouped', {}):
                    descs = file[db][tbl]['ungrouped'][u_key]['details']['description']
                    count += sum(1 for desc in descs.values() if desc is None)
        return count

    # 📊 Count before update
    before_none = count_none_descriptions(file)
    print(f"\n🔍 Number of `None` descriptions before update: {before_none}\n")

    # Track printed checks
    printed = 0
    max_print = 10
    updated_count = 0

    # 🟢 Grouped updates
    for database in file:
        for table in file[database]:
            for template in file[database][table].get('grouped', {}):
                for t_index, item in enumerate(file[database][table]['grouped'][template]):
                    for column_name in item['details']['description']:
                        if item['details']['description'][column_name] is None:
                            key = (database, table, template, t_index, column_name)
                            if key in grouped_lookup:
                                description = grouped_lookup[key]
                                file[database][table]['grouped'][template][t_index]['details']['description'][column_name] = description
                                updated_count += 1
                                if printed < max_print:
                                    print(f"[Grouped] Inserted at file['{database}']['{table}']['grouped']['{template}'][{t_index}]['details']['description']['{column_name}']:")
                                    print(f"  ➜ {description}\n")
                                    printed += 1

    # 🔵 Ungrouped updates
    for database in file:
        for table in file[database]:
            for ungrouped_key, content in file[database][table].get('ungrouped', {}).items():
                for column_name in content['details']['description']:
                    if content['details']['description'][column_name] is None:
                        key = (database, table, ungrouped_key, column_name)
                        if key in ungrouped_lookup:
                            description = ungrouped_lookup[key]
                            file[database][table]['ungrouped'][ungrouped_key]['details']['description'][column_name] = description
                            updated_count += 1
                            if printed < max_print:
                                print(f"[Ungrouped] Inserted at file['{database}']['{table}']['ungrouped']['{ungrouped_key}']['details']['description']['{column_name}']:")
                                print(f"  ➜ {description}\n")
                                printed += 1

# 📊 Count after update
after_none = count_none_descriptions(file)

# ✅ Summary
print("\n✅ Update Summary:")
print(f"  🔧 Total columns updated: {updated_count}")
print(f"  ❗ Remaining `None` descriptions: {after_none}")
print(f"  🕵️ Previously `None`: {before_none}")
print(f"  🧮 Final total filled in: {before_none - after_none}")



🔍 Number of `None` descriptions before update: 71704

[Grouped] Inserted at file['NEW_YORK']['NEW_YORK']['grouped']['TLC_GREEN_TRIPS_201{variable0}.json'][0]['details']['description']['ehail_fee']:
  ➜ The ehail_fee represents an additional charge applied to electronic hails for rides booked through a mobile application or dispatch system.

[Grouped] Inserted at file['NEW_YORK']['NEW_YORK']['grouped']['TLC_GREEN_TRIPS_201{variable0}.json'][0]['details']['description']['time_between_service']:
  ➜ This column represents the time duration in seconds between the end of one taxi trip and the start of the next service trip for a given vehicle, providing insights into operational efficiency and wait times.

[Grouped] Inserted at file['NEW_YORK']['NEW_YORK']['grouped']['TLC_GREEN_TRIPS_201{variable0}.json'][0]['details']['description']['distance_between_service']:
  ➜ The distance_between_service column records the geographic distance covered during the service period of a trip, measured in 

### Generating the prompts for the columns

In [6]:
import random

def open_ai_call(client, model, system_prompt, user_prompt, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

def clean_and_parse_result(result):
    cleaned = re.sub(r"^```(?:json)?\n?", "", result.strip())
    cleaned = re.sub(r"\n?```$", "", cleaned.strip())

    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        pass

    try:
        return ast.literal_eval(cleaned)
    except Exception:
        pass

    match = re.search(r"(\[.*\]|\{.*\})", cleaned, re.DOTALL)
    if match:
        snippet = match.group(0)
        try:
            return json.loads(snippet)
        except json.JSONDecodeError:
            try:
                return ast.literal_eval(snippet)
            except Exception as e:
                print("⚠️ Failed to parse fallback snippet")
                print(snippet)
                raise e

    raise ValueError("⚠️ Could not parse cleaned result.")

def count_tokens(text, model="gpt-4o-mini-2024-07-18"):
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))

def create_column_description_prompt(
    target_column_strs,  # now expects a list
    table_description,
    other_column_descriptions={},
    database=None,
    table=None,
    template=None,
    ungrouped_file=None,
    model="gpt-4o-mini-2024-07-18",
    token_limit=500,
    allow_sampling_if_too_long=True,
    max_columns=25
):
    def parse_target(target):
        if "::" not in target:
            raise ValueError(f"Expected format 'column_name::type', got: {target}")
        return target.split("::", 1)

    def format_existing_columns(columns_dict):
        return [f"- {col}: {desc}" for col, desc in columns_dict.items()]

    def format_column_names_only(col_names):
        return [f"- {col}" for col in col_names]

    def format_target_columns(targets):
        return "\n".join([f"- {name} ({typ})" for name, typ in targets])

    parsed_targets = [parse_target(target) for target in target_column_strs]

    clean_descriptions = {
        k: v for k, v in (other_column_descriptions or {}).items()
        if v is not None and k not in [name for name, _ in parsed_targets]
    }

    all_other_column_names = [
        k for k in (other_column_descriptions or {}) if k not in [name for name, _ in parsed_targets]
    ]

    def build_prompt(existing_lines, subset_note):
        columns_block = "\n".join(existing_lines) if existing_lines else "None"

        if template is not None:
            intro = (
                f"You are working with columns from the `{template}` table, "
                f"which belongs to `{table}` in the `{database}` database."
            )
        elif ungrouped_file is not None:
            intro = (
                f"You are working with columns from a standalone table `{ungrouped_file}` "
                f"in `{table}` of the `{database}` database."
            )
        else:
            intro = f"You are working with columns in `{table}` of the `{database}` database."

        targets_block = format_target_columns(parsed_targets)

        return f"""{intro}

- Table Description: {table_description}

{subset_note}

- Existing Columns:
{columns_block}

- Target Columns (to describe):
{targets_block}
"""

    # === CASE 1: No descriptions, only names ===
    if not clean_descriptions and all_other_column_names:
        fallback_sample = random.sample(all_other_column_names, min(max_columns, len(all_other_column_names)))
        fallback_lines = format_column_names_only(fallback_sample)

        note = "Note: Only column names are shown, as no descriptions were available."
        if len(fallback_sample) < len(all_other_column_names):
            note += " Not all columns are listed."

        return build_prompt(
            fallback_lines,
            note
        )

    # === CASE 2: Try full list ===
    col_items = list(clean_descriptions.items())
    random.shuffle(col_items)
    col_items = col_items[:max_columns]

    full_lines = format_existing_columns(dict(col_items))
    full_prompt = build_prompt(
        full_lines,
        "Note: All columns of the table are listed below."
    )
    if count_tokens(full_prompt, model=model) <= token_limit:
        return full_prompt

    # === CASE 3: Shrink to fit ===
    if not allow_sampling_if_too_long or not col_items:
        return build_prompt([], "Note: No other columns are shown due to token limits.")

    lo, hi = 1, len(col_items)
    best_prompt = None

    while lo <= hi:
        mid = (lo + hi) // 2
        sampled_dict = dict(col_items[:mid])
        sampled_lines = format_existing_columns(sampled_dict)
        prompt = build_prompt(
            sampled_lines,
            "Note: Only a subset of the table’s columns is provided below."
        )
        if count_tokens(prompt, model=model) <= token_limit:
            best_prompt = prompt
            lo = mid + 1
        else:
            hi = mid - 1

    return best_prompt or build_prompt([], "Note: No other columns are shown due to token limits.")


In [7]:
count_no_desc = 0
count_desc = 0

grouped_print = 0
ungrouped_print = 0

# First, count total number of columns to initialize progress bar
total_columns = 0
for database in file:
    for table in file[database]:
        for template in file[database][table].get('grouped', {}):
            for t in file[database][table]['grouped'][template]:
                total_columns += len(t['details']['columns'])
        for t in file[database][table].get('ungrouped', {}):
            total_columns += len(file[database][table]['ungrouped'][t]['details']['columns'])

# Begin processing with progress bar
with tqdm(total=total_columns, desc="Generating column prompts") as pbar:
    for database in file:
        for table in file[database]:

            # === GROUPED ===
            for template in file[database][table].get('grouped', {}):
                for t_index, group_entry in enumerate(file[database][table]['grouped'][template]):
                    column_names = group_entry['details']['columns']
                    column_descriptions = group_entry['details']['description']
                    group_entry['details']['prompts'] = {}

                    missing_columns = {
                        name: typ for name, typ in column_names.items()
                        if column_descriptions[name] is None
                    }
                    described_columns = set(column_names) - set(missing_columns)

                    # Handle described columns
                    for col in described_columns:
                        group_entry['details']['prompts'][col] = None
                        count_desc += 1
                        pbar.update(1)

                    # Handle missing descriptions in batches of 10
                    missing_items = list(missing_columns.items())
                    for i in range(0, len(missing_items), 10):
                        batch = missing_items[i:i+10]
                        target_column_strs = [f"{name}::{typ}" for name, typ in batch]

                        col_prompt = create_column_description_prompt(
                            target_column_strs=target_column_strs,
                            table_description=group_entry['description'],
                            other_column_descriptions=column_descriptions,
                            database=database,
                            table=table,
                            template=template,
                            token_limit=500
                        )

                        if grouped_print == 0:
                            print(col_prompt)
                            grouped_print += 1

                        for name, _ in batch:
                            group_entry['details']['prompts'][name] = col_prompt
                            count_no_desc += 1
                            pbar.update(1)


            # === UNGROUPED ===
            for ungrouped_key, ungrouped_entry in file[database][table].get('ungrouped', {}).items():
                column_names = ungrouped_entry['details']['columns']
                column_descriptions = ungrouped_entry['details']['description']
                ungrouped_entry['details']['prompts'] = {}

                missing_columns = {
                    name: typ for name, typ in column_names.items()
                    if column_descriptions[name] is None
                }
                described_columns = set(column_names) - set(missing_columns)

                for col in described_columns:
                    ungrouped_entry['details']['prompts'][col] = None
                    count_desc += 1
                    pbar.update(1)

                missing_items = list(missing_columns.items())
                for i in range(0, len(missing_items), 10):
                    batch = missing_items[i:i+10]
                    target_column_strs = [f"{name}::{typ}" for name, typ in batch]

                    col_prompt = create_column_description_prompt(
                        target_column_strs=target_column_strs,
                        table_description=ungrouped_entry['description'],
                        other_column_descriptions=column_descriptions,
                        database=database,
                        table=table,
                        ungrouped_file=ungrouped_key,
                        token_limit=500
                    )

                    if ungrouped_print == 0:
                        print(col_prompt)
                        ungrouped_print += 1

                    for name, _ in batch:
                        ungrouped_entry['details']['prompts'][name] = col_prompt
                        count_no_desc += 1
                        pbar.update(1)


print("📝 Columns without description:", count_no_desc)
print("✅ Columns with description:", count_desc)


Generating column prompts: 100%|██████████| 134095/134095 [00:00<00:00, 1066982.33it/s]

📝 Columns without description: 0
✅ Columns with description: 134095


### Multiple columns at the time code

In [8]:
with open("/Users/paolocadei/Documents/Masters/Thesis/Spider2/api_keys/api_keys.json", "r") as f:
    api_keys = dict(json.load(f))

OPENAI_API_KEY =  api_keys["open_ai_key"]
OPENAI_API_URL = "https://api.openai.com/v1/chat/completions"
MODEL = "gpt-4o-mini-2024-07-18" #'gpt-4.1-mini-2025-04-14'  "gpt-4o-mini-2024-07-18"
BATCH_SIZE = 15
MAX_COUNT = None

HEADERS = {
    "Authorization": f"Bearer {OPENAI_API_KEY}",
    "Content-Type": "application/json"
}

# === LIMITS (75% SAFETY) ===
ESTIMATED_TOKENS_PER_CALL = 1000
MAX_REQUESTS_PER_MIN = 500
MAX_TOKENS_PER_MIN = 120_000
MAX_REQUESTS_PER_DAY = 10_000
MAX_TOKENS_PER_DAY = 2_000_000

# === PROMPT ===
system_prompt = """You are generating metadata for **multiple columns** in a database table.

Your task is to describe what each column contains and how users might refer to it, based on the table's description and other columns.

For each column:
- Write a concise, clear sentence describing its **meaning and purpose**.
- Then provide **10 relevant keywords or phrases** capturing its semantic content (used for semantic search and matching).
- Focus on real-world concepts. Avoid redundancy or just repeating the column name.

**Output format**:
{
  "column_name_1": {
    "description": "...",
    "keywords": ["...", "..."]
  },
  "column_name_2": {
    ...
  }
}"""


# === UTILITIES ===
def log_jsonl(path, entry):
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(entry) + "\n")

def canonical_hash(meta):
    canonical_meta = {
        "database": meta.get("database"),
        "table": meta.get("table"),
        "column": meta.get("column"),
        "template": meta.get("template", None),
        "ungrouped_key": meta.get("ungrouped_key", None),
        "entry_index": meta.get("entry_index", None)
    }
    return hashlib.md5(json.dumps(canonical_meta, sort_keys=True).encode()).hexdigest()

def load_hashes_from_jsonl(file_path):
    hashes = set()
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                entry = json.loads(line)
                hashes.add(canonical_hash(entry))
    except FileNotFoundError:
        pass
    return hashes

async def openai_call(session, prompt, retries=10, wait_time=60):
    data = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 2000
    }

    for attempt in range(retries):
        try:
            async with session.post(OPENAI_API_URL, headers=HEADERS, json=data) as resp:
                if resp.status == 429:
                    raise aiohttp.ClientResponseError(
                        status=429, request_info=resp.request_info, history=resp.history, message="Rate limit hit"
                    )
                resp.raise_for_status()
                result = await resp.json()
                return result["choices"][0]["message"]["content"]

        except aiohttp.ClientResponseError as e:
            if e.status == 429 or 500 <= e.status < 600:
                if attempt < retries - 1:
                    print(f"[Retrying] Attempt {attempt + 1}/{retries} due to status {e.status}. Waiting {wait_time}s...")
                    await asyncio.sleep(wait_time)
                else:
                    print(f"[ERROR] Max retries reached for prompt. Last error: {e}. Waiting 120s before continuing...")
                    await asyncio.sleep(120)
                    return await openai_call(session, prompt)  # Recursively retry once more after delay
            else:
                print(f"[ERROR] Unhandled response error: {e}")
                raise
        except Exception as e:
            print(f"[ERROR] Unexpected exception: {e}")
            raise

async def process_prompt(session, col_prompt, columns, meta_template, entry_ref, desc_key, keywords_key):
    try:
        response = await openai_call(session, col_prompt)
        result = json.loads(response)

        # 1. Log each column individually (existing behavior)
        for column_name in columns:
            if column_name in result:
                entry_ref[desc_key][column_name] = result[column_name]["description"]
                entry_ref[keywords_key][column_name] = result[column_name]["keywords"]

                meta = dict(meta_template)
                meta["column"] = column_name

                log_jsonl("2_successes_multiple.jsonl", {
                    **meta,
                    "status": "success",
                    "prompt": col_prompt,
                    "response": result[column_name]
                })

        # 2. Log full raw response and summary of batch
        log_jsonl("2_batch_successes.jsonl", {
            "columns": columns,
            "meta": {k: v for k, v in meta_template.items() if k != "column"},
            "prompt": col_prompt,
            "response": result,
            "status": "batch_success"
        })

        # 3. Optional: Save full raw response separately for archival
        log_jsonl("2_raw_responses.jsonl", {
            "prompt": col_prompt,
            "response": result
        })

    except Exception as e:
        for column_name in columns:
            meta = dict(meta_template)
            meta["column"] = column_name
            log_jsonl("failures.jsonl", {
                **meta, "status": "error", "prompt": col_prompt, "error": str(e)}
            )


async def main():
    while True:
        try:
            success_hashes = load_hashes_from_jsonl("2_successes_multiple.jsonl")
            tasks = []
            count = 0
            processed_prompts = set()

            async with aiohttp.ClientSession() as session:
                for database in file:
                    for table in file[database]:
                        # === GROUPED ===
                        for template in file[database][table].get('grouped', {}):
                            for t_index, group_entry in enumerate(file[database][table]['grouped'][template]):
                                group_entry['details']['keywords'] = {}

                                prompt_to_columns = defaultdict(list)
                                prompt_to_meta = {}
                                prompt_to_entry_ref = {}

                                for column_name in group_entry['details']['columns']:
                                    prompt = group_entry['details']['prompts'].get(column_name)
                                    if prompt is None or group_entry['details']['description'].get(column_name) is not None:
                                        continue

                                    meta = {
                                        "database": database,
                                        "table": table,
                                        "column": column_name,
                                        "template": template,
                                        "ungrouped_key": None,
                                        "entry_index": t_index
                                    }

                                    if canonical_hash(meta) in success_hashes:
                                        continue

                                    prompt_to_columns[prompt].append(column_name)
                                    prompt_to_meta[prompt] = meta
                                    prompt_to_entry_ref[prompt] = group_entry['details']

                                for prompt, columns in prompt_to_columns.items():
                                    if prompt in processed_prompts:
                                        continue
                                    processed_prompts.add(prompt)
                                    task = process_prompt(
                                        session=session,
                                        col_prompt=prompt,
                                        columns=columns,
                                        meta_template=prompt_to_meta[prompt],
                                        entry_ref=prompt_to_entry_ref[prompt],
                                        desc_key="description",
                                        keywords_key="keywords"
                                    )
                                    tasks.append(task)
                                    count += len(columns)

                        # === UNGROUPED ===
                        for ungrouped_key, ungrouped_entry in file[database][table].get('ungrouped', {}).items():
                            ungrouped_entry['details']['keywords'] = {}

                            prompt_to_columns = defaultdict(list)
                            prompt_to_meta = {}
                            prompt_to_entry_ref = {}

                            for column_name in ungrouped_entry['details']['columns']:
                                prompt = ungrouped_entry['details']['prompts'].get(column_name)
                                if prompt is None or ungrouped_entry['details']['description'].get(column_name) is not None:
                                    continue

                                meta = {
                                    "database": database,
                                    "table": table,
                                    "column": column_name,
                                    "template": None,
                                    "ungrouped_key": ungrouped_key,
                                    "entry_index": None
                                }

                                if canonical_hash(meta) in success_hashes:
                                    continue

                                prompt_to_columns[prompt].append(column_name)
                                prompt_to_meta[prompt] = meta
                                prompt_to_entry_ref[prompt] = ungrouped_entry['details']

                            for prompt, columns in prompt_to_columns.items():
                                if prompt in processed_prompts:
                                    continue
                                processed_prompts.add(prompt)
                                task = process_prompt(
                                    session=session,
                                    col_prompt=prompt,
                                    columns=columns,
                                    meta_template=prompt_to_meta[prompt],
                                    entry_ref=prompt_to_entry_ref[prompt],
                                    desc_key="description",
                                    keywords_key="keywords"
                                )
                                tasks.append(task)
                                count += len(columns)

                print(f"Queued {len(tasks)} prompts to process...")

                requests_this_minute = 0
                tokens_this_minute = 0
                requests_today = 0
                tokens_today = 0
                minute_start = time.time()

                total_prompts = len(tasks)
                batch_indices = list(range(0, total_prompts, BATCH_SIZE))

                with tqdm(total=total_prompts, desc="Total Prompts", unit="prompt") as prompt_bar:
                    with tqdm(total=len(batch_indices), desc="Batches", unit="batch") as batch_bar:
                        for i in batch_indices:
                            now = time.time()
                            elapsed = now - minute_start

                            if elapsed >= 60:
                                requests_this_minute = 0
                                tokens_this_minute = 0
                                minute_start = now

                            batch = tasks[i:i+BATCH_SIZE]

                            projected_requests_min = requests_this_minute + len(batch)
                            projected_tokens_min = tokens_this_minute + len(batch) * ESTIMATED_TOKENS_PER_CALL
                            projected_requests_day = requests_today + len(batch)
                            projected_tokens_day = tokens_today + len(batch) * ESTIMATED_TOKENS_PER_CALL

                            if projected_requests_day > MAX_REQUESTS_PER_DAY or projected_tokens_day > MAX_TOKENS_PER_DAY:
                                print(f"[STOP] Daily request or token limit reached ({requests_today} requests, {tokens_today} tokens used). Halting.")
                                return

                            if projected_requests_min > MAX_REQUESTS_PER_MIN or projected_tokens_min > MAX_TOKENS_PER_MIN:
                                wait_time = 60 - elapsed
                                print(f"[WAIT] Sleeping {wait_time:.2f}s to stay within RPM/TPM limits...")
                                await asyncio.sleep(wait_time)
                                requests_this_minute = 0
                                tokens_this_minute = 0
                                minute_start = time.time()

                            await asyncio.gather(*batch)

                            requests_this_minute += len(batch)
                            tokens_this_minute += len(batch) * ESTIMATED_TOKENS_PER_CALL
                            requests_today += len(batch)
                            tokens_today += len(batch) * ESTIMATED_TOKENS_PER_CALL

                            prompt_bar.update(len(batch))
                            batch_bar.update(1)

            break  # All good, exit loop

        except Exception as e:
            print(f"[RETRY LOOP] An error occurred: {e}. Waiting 120s before retrying...")
            await asyncio.sleep(120)

# === RUN ASYNC IN JUPYTER ===
nest_asyncio.apply()
asyncio.get_event_loop().run_until_complete(main())


Queued 0 prompts to process...


Batches: 0batch [00:00, ?batch/s]rompt/s]
Total Prompts: 0prompt [00:00, ?prompt/s]


### Saving the columns with descriptions

In [9]:
with open('2_final_structure_with_descriptions.json', 'w') as f:
    json.dump(file, f, indent=4)

In [121]:
with open('2_final_structure_with_descriptions.json', 'r') as f:
    file = json.load(f)

count_none_descriptions(file)

0

### Inserting keywords created before when creating the column descriptions

In [123]:
import json

def safe_load_jsonl_lines(filepath):
    lines = []
    bad_lines = 0
    with open(filepath, "r") as f:
        for i, line in enumerate(f):
            try:
                lines.append(json.loads(line))
            except json.JSONDecodeError:
                bad_lines += 1
                print(f"[Warning] Invalid JSON at {filepath}, line {i+1}")
    return lines, bad_lines

def inserting_keywords(file, responses):
    total_added_keywords = 0
    total_missing_keywords = 0
    total_bad_lines = 0
    total_already_present = 0
    per_file_stats = []

    # 🧮 Count already existing keywords (non-None)
    for database in file:
        for table in file[database]:
            # Grouped
            for template in file[database][table].get('grouped', {}):
                for item in file[database][table]['grouped'][template]:
                    columns = item['details']['columns']
                    keywords = item['details'].get('keywords', {})
                    for col in columns:
                        if keywords.get(col) is not None:
                            total_already_present += 1
            # Ungrouped
            for content in file[database][table].get('ungrouped', {}).values():
                columns = content['details']['columns']
                keywords = content['details'].get('keywords', {})
                for col in columns:
                    if keywords.get(col) is not None:
                        total_already_present += 1

    # 🔁 Process each response file
    for response in responses:
        added_keywords = 0
        bad_lines = 0

        success_lines, bad_count = safe_load_jsonl_lines(response)
        bad_lines += bad_count
        total_bad_lines += bad_count

        # Build lookups
        ungrouped_lookup = {}
        grouped_lookup = {}

        for entry in success_lines:
            if entry['status'] != 'success':
                continue
            if entry['ungrouped_key'] is not None:
                ungrouped_lookup[(entry['database'], entry['table'], entry['ungrouped_key'], entry['column'])] = entry['response']['keywords']
            else:
                grouped_lookup[(entry['database'], entry['table'], entry['template'], entry['entry_index'], entry['column'])] = entry['response']['keywords']

        # 🟢 Grouped updates
        for database in file:
            for table in file[database]:
                for template in file[database][table].get('grouped', {}):
                    for t_index, item in enumerate(file[database][table]['grouped'][template]):
                        if 'keywords' not in item['details']:
                            item['details']['keywords'] = {k: None for k in item['details']['columns'].keys()}
                        for col in item['details']['columns'].keys():
                            current_value = item['details']['keywords'].get(col)
                            key = (database, table, template, t_index, col)
                            if key in grouped_lookup:
                                if current_value is None:
                                    added_keywords += 1
                                item['details']['keywords'][col] = grouped_lookup[key]

        # 🔵 Ungrouped updates
        for database in file:
            for table in file[database]:
                for ungrouped_key, content in file[database][table].get('ungrouped', {}).items():
                    if 'keywords' not in content['details']:
                        content['details']['keywords'] = {k: None for k in content['details']['columns'].keys()}
                    for col in content['details']['columns'].keys():
                        current_value = content['details']['keywords'].get(col)
                        key = (database, table, ungrouped_key, col)
                        if key in ungrouped_lookup:
                            if current_value is None:
                                added_keywords += 1
                            content['details']['keywords'][col] = ungrouped_lookup[key]

        total_added_keywords += added_keywords
        per_file_stats.append((response, added_keywords, bad_lines))

    # 🔍 Count remaining missing keywords
    missing_preview = []
    for database in file:
        for table in file[database]:
            # Grouped
            for template in file[database][table].get('grouped', {}):
                for idx, item in enumerate(file[database][table]['grouped'][template]):
                    columns = item['details']['columns']
                    keywords = item['details'].get('keywords', {})
                    for col in columns:
                        kw = keywords.get(col)
                        if kw is None:
                            total_missing_keywords += 1
                            if len(missing_preview) < 5:
                                missing_preview.append(("grouped", database, table, template, idx, col))
            # Ungrouped
            for u_key, content in file[database][table].get('ungrouped', {}).items():
                columns = content['details']['columns']
                keywords = content['details'].get('keywords', {})
                for col in columns:
                    kw = keywords.get(col)
                    if kw is None:
                        total_missing_keywords += 1
                        if len(missing_preview) < 5:
                            missing_preview.append(("ungrouped", database, table, u_key, None, col))

    # 📊 Reporting
    print(f"📌 Keywords already present before insertion: {total_already_present}")
    for filename, added, bad in per_file_stats:
        print(f"📂 {filename}:")
        print(f"   ➕ Keywords added: {added}")
        print(f"   ⚠️  Corrupted lines: {bad}")
    print(f"\n✅ Total keywords added: {total_added_keywords}")
    print(f"❌ Keywords still missing: {total_missing_keywords}")
    print(f"⚠️ Total corrupted JSON lines skipped: {total_bad_lines}")

    if missing_preview:
        print("\n🔍 Example missing columns:")
        for kind, db, tbl, tmpl_or_key, idx, col in missing_preview:
            if kind == "grouped":
                print(f"  ➤ Grouped | DB: {db} | Table: {tbl} | Template: {tmpl_or_key} | Row: {idx} | Column: {col}")
            else:
                print(f"  ➤ Ungrouped | DB: {db} | Table: {tbl} | Entry: {tmpl_or_key} | Column: {col}")

    return file


# Example usage
responses = ['2_successes.jsonl', '2_successes_multiple.jsonl', '2_keyword_results.jsonl']
file = inserting_keywords(file, responses)

📌 Keywords already present before insertion: 134095
📂 2_successes.jsonl:
   ➕ Keywords added: 0
   ⚠️  Corrupted lines: 0
📂 2_successes_multiple.jsonl:
   ➕ Keywords added: 0
   ⚠️  Corrupted lines: 0
📂 2_keyword_results.jsonl:
   ➕ Keywords added: 0
   ⚠️  Corrupted lines: 0

✅ Total keywords added: 0
❌ Keywords still missing: 0
⚠️ Total corrupted JSON lines skipped: 0


### Column keywords creation for the missing ones

In [124]:
import asyncio
import aiohttp
import json
import os
import time
from tqdm.asyncio import tqdm

# === CONFIGURATION ===
with open("/Users/paolocadei/Documents/Masters/Thesis/Spider2/api_keys/api_keys.json", "r") as f:
    api_keys = dict(json.load(f))

OPENAI_API_KEY = api_keys["open_ai_key"]
OPENAI_API_URL = "https://api.openai.com/v1/chat/completions"
MODEL = "gpt-4o-mini-2024-07-18"
MAX_COUNT = None
TEMP_FILE = "2_keyword_results.json.tmp"
FINAL_FILE = "2_keyword_results.jsonl"
FAILED_FILE = "2_keyword_failed_requests.jsonl"
REQUEST_SAVE_INTERVAL = 1 # SET TO 10 IF THERE ARE MORE THAN 10
MAX_CONCURRENT_REQUESTS = 1

ESTIMATED_TOKENS_PER_CALL = 2000
MAX_REQUESTS_PER_MIN = 100
MAX_TOKENS_PER_MIN = 75000
MAX_REQUESTS_PER_DAY = 5000
MAX_TOKENS_PER_DAY = 2000000

HEADERS = {
    "Authorization": f"Bearer {OPENAI_API_KEY}",
    "Content-Type": "application/json"
}

system_prompt = """You are helping build an NL2SQL system that translates natural language questions into SQL queries.

Given multiple columns in a database table, your task is to return a dictionary that maps each column name to a list of **10 relevant keywords or phrases** that describe its semantic content.

These keywords will be used to match user questions to the correct columns, so focus on how users might naturally refer to each column. Avoid generic terms and do not repeat the column name unless it’s a common alias.

**Output format**:
{
  "column_name_1": ["keyword 1", ..., "keyword 10"],
  "column_name_2": ["keyword 1", ..., "keyword 10"]
}
"""

# === Prompt builder ===
def build_batch_prompt(database, table, template, column_dict, table_description):
    formatted_columns = "\n".join([f"- {col}: {desc}" for col, desc in column_dict.items()])
    return (
        f"You are working on the table '{template}' in '{table}' from the database '{database}'.\n"
        f"The table is described as: {table_description}\n\n"
        f"The following columns need keywords:\n{formatted_columns}\n\n"
        f"Return a dictionary mapping each column to 10 relevant keywords."
    )

# === OpenAI call with retry and pacing ===
semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
requests_this_minute = 0
tokens_this_minute = 0
requests_today = 0
tokens_today = 0
minute_start = time.time()

async def fetch_keywords(session, prompt, column_ids, retries=5, wait_time=60):
    global requests_this_minute, tokens_this_minute, requests_today, tokens_today, minute_start
    payload = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.3
    }

    for attempt in range(retries):
        async with semaphore:
            now = time.time()
            if now - minute_start >= 60:
                requests_this_minute = 0
                tokens_this_minute = 0
                minute_start = now

            if (requests_this_minute + 1 > MAX_REQUESTS_PER_MIN or
                tokens_this_minute + ESTIMATED_TOKENS_PER_CALL > MAX_TOKENS_PER_MIN):
                sleep_time = 60 - (now - minute_start)
                print(f"[WAIT] Sleeping {sleep_time:.1f}s to stay under rate limit...")
                await asyncio.sleep(sleep_time)
                requests_this_minute = 0
                tokens_this_minute = 0
                minute_start = time.time()

            try:
                async with session.post(OPENAI_API_URL, headers=HEADERS, json=payload) as response:
                    if response.status == 429:
                        raise aiohttp.ClientResponseError(
                            status=429, request_info=response.request_info, history=response.history, message="Rate limit hit"
                        )
                    response.raise_for_status()
                    result = await response.json()
                    content = result["choices"][0]["message"]["content"]
                    requests_this_minute += 1
                    tokens_this_minute += ESTIMATED_TOKENS_PER_CALL
                    requests_today += 1
                    tokens_today += ESTIMATED_TOKENS_PER_CALL
                    return column_ids, content

            except aiohttp.ClientResponseError as e:
                if e.status == 429 or 500 <= e.status < 600:
                    print(f"[Retry] {e.status} — sleeping {wait_time}s (attempt {attempt + 1})")
                    await asyncio.sleep(wait_time)
                    continue
                else:
                    print(f"[Error] Non-retryable: {e.status}")
                    break
            except Exception as e:
                print(f"[Exception] {e}")
                break

    return column_ids, "null_response_retry_limit_reached"

# === Save helpers ===
def save_tmp(jsonl_lines):
    with open(TEMP_FILE, "w") as f:
        f.write("\n".join(jsonl_lines) + "\n")
    #print(f"💾 Temp file {TEMP_FILE} saved.")

def flush_tmp_to_final():
    if os.path.exists(TEMP_FILE):
        with open(TEMP_FILE, "r") as tmp_f:
            tmp_data = tmp_f.read()
        with open(FINAL_FILE, "a") as final_f:
            final_f.write(tmp_data)
        os.remove(TEMP_FILE)
        #print(f"📤 Flushed temp file to {FINAL_FILE} and deleted {TEMP_FILE}")

def log_failed(column_ids, raw_content):
    with open(FAILED_FILE, "a") as f:
        f.write(json.dumps({
            "column_ids": column_ids,
            "response": raw_content[:300]  # Limit long output
        }) + "\n")
    print(f"❌ Logged failure for {column_ids} to {FAILED_FILE}")

# === Load processed columns from final file ===
def load_processed_column_ids():
    if not os.path.exists(FINAL_FILE):
        return set()
    processed = set()
    with open(FINAL_FILE, "r") as f:
        for line in f:
            try:
                entry = json.loads(line)
                key = (entry["database"], entry["table"], entry["column"])
                processed.add(key)
            except Exception:
                continue
    print(f"🔁 Found {len(processed)} previously processed columns in {FINAL_FILE}")
    return processed

async def process_keywords(file):
    processed_columns = load_processed_column_ids()
    jsonl_buffer = []
    columns_processed = 0
    request_batch_buffer = []
    total_columns_missing = 0
    total_prompts_sent = 0

    async with aiohttp.ClientSession() as session:
        tasks = []

        for database in file:
            for table in file[database]:
                # GROUPED
                for template in file[database][table].get('grouped', {}):
                    for entry_index, entry in enumerate(file[database][table]['grouped'][template]):
                        table_description = entry.get('description', '')
                        columns = entry['details']['columns']
                        keywords = entry['details'].get('keywords', {})
                        
                        missing = {
                            column: desc for column, desc in columns.items()
                            if keywords.get(column) is None
                        }

                        total_columns_missing += len(missing)
                        for i in range(0, len(missing), 10):
                            batch = dict(list(missing.items())[i:i+10])
                            if not batch:
                                continue
                            prompt = build_batch_prompt(database, table, template, batch, table_description)
                            column_ids = [("grouped", database, table, template, entry_index, col) for col in batch]
                            tasks.append(fetch_keywords(session, prompt, column_ids))
                            total_prompts_sent += 1

                # UNGROUPED
                for ungrouped_key, content in file[database][table].get('ungrouped', {}).items():
                    table_description = content.get('description', '')
                    columns = content['details']['columns']
                    keywords = content['details'].get('keywords', {})
                    missing = {
                            column: desc for column, desc in columns.items()
                            if keywords.get(column) is None
                        }
                    
                    total_columns_missing += len(missing)
                    for i in range(0, len(missing), 10):
                        batch = dict(list(missing.items())[i:i+10])
                        if not batch:
                            continue
                        prompt = build_batch_prompt(database, table, ungrouped_key, batch, table_description)
                        column_ids = [("ungrouped", database, table, ungrouped_key, col) for col in batch]
                        tasks.append(fetch_keywords(session, prompt, column_ids))
                        total_prompts_sent += 1

        # 🖨️ Print summary before sending requests
        print(f"📌 Total missing columns to process: {total_columns_missing}")
        print(f"📬 Total prompts to send (batches of ≤10): {total_prompts_sent}")
        print(f"🔄 Sending {len(tasks)} requests to OpenAI...")

        for coro in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="🔍 Fetching keywords"):
            column_ids, content = await coro
            request_batch_buffer.append((column_ids, content))

            if len(request_batch_buffer) >= REQUEST_SAVE_INTERVAL:
                jsonl_lines = []
                for column_ids, content in request_batch_buffer:

                    try:
                        parsed = json.loads(content)

                    except Exception:
                        log_failed(column_ids, content)
                        continue
                    for column_id in column_ids:

                        if column_id[0] == "ungrouped":
                            _, db, tbl, key, col = column_id
                            keywords = parsed.get(col)

                            if keywords:
                                prompt = build_batch_prompt(db, tbl, key, {col: ""}, "")
                                jsonl_lines.append(json.dumps({
                                    "status": "success",
                                    "database": db,
                                    "table": tbl,
                                    "template": None,
                                    "ungrouped_key": key,
                                    "entry_index": None,
                                    "column": col,
                                    "prompt": prompt,
                                    "response": {"keywords": keywords}
                                }))
                        else:
                            _, db, tbl, template, idx, col = column_id
                            keywords = parsed.get(col)
                            if keywords:
                                prompt = build_batch_prompt(db, tbl, template, {col: ""}, "")
                                jsonl_lines.append(json.dumps({
                                    "status": "success",
                                    "database": db,
                                    "table": tbl,
                                    "template": template,
                                    "ungrouped_key": None,
                                    "entry_index": idx,
                                    "column": col,
                                    "prompt": prompt,
                                    "response": {"keywords": keywords}
                                }))

                save_tmp(jsonl_lines)
                flush_tmp_to_final()
                request_batch_buffer = []

        print("✅ All requests processed.")

# === Run ===
if __name__ == "__main__":
    asyncio.run(process_keywords(file))

🔁 Found 32900 previously processed columns in 2_keyword_results.jsonl
📌 Total missing columns to process: 0
📬 Total prompts to send (batches of ≤10): 0
🔄 Sending 0 requests to OpenAI...


🔍 Fetching keywords: 0it [00:00, ?it/s]

✅ All requests processed.


In [125]:
# Inserting the new values we have generated
responses = ['2_keyword_results.jsonl']

file = inserting_keywords(file, responses)

📌 Keywords already present before insertion: 134095
📂 2_keyword_results.jsonl:
   ➕ Keywords added: 0
   ⚠️  Corrupted lines: 0

✅ Total keywords added: 0
❌ Keywords still missing: 0
⚠️ Total corrupted JSON lines skipped: 0


In [126]:
with open('2_final_structure_all.json', 'w') as f:
    json.dump(file, f, indent=4)

### Checking if all columns have the keywords

In [127]:
def count_keywords_status(file, print_first_missing=True):
    count_with_keywords = 0
    count_missing_keywords = 0
    first_missing_info = None

    for db in file:
        for tbl in file[db]:
            # Grouped
            for tmpl in file[db][tbl].get('grouped', {}):
                for idx, t in enumerate(file[db][tbl]['grouped'][tmpl]):
                    columns = t['details']['columns']
                    keywords = t['details'].get('keywords', {})
                    for col in columns:
                        kw = keywords.get(col)
                        if kw is None:
                            count_missing_keywords += 1
                            if print_first_missing and first_missing_info is None:
                                first_missing_info = ("grouped", db, tbl, tmpl, idx, col)
                        else:
                            count_with_keywords += 1

            # Ungrouped
            for u_key in file[db][tbl].get('ungrouped', {}):
                columns = file[db][tbl]['ungrouped'][u_key]['details']['columns']
                keywords = file[db][tbl]['ungrouped'][u_key]['details'].get('keywords', {})
                for col in columns:
                    kw = keywords.get(col)
                    if kw is None:
                        count_missing_keywords += 1
                        if print_first_missing and first_missing_info is None:
                            first_missing_info = ("ungrouped", db, tbl, u_key, col)
                    else:
                        count_with_keywords += 1

    # Print first missing location if needed
    if print_first_missing and first_missing_info:
        print("🔍 First missing keyword found at:")
        if first_missing_info[0] == "grouped":
            _, db, tbl, tmpl, idx, col = first_missing_info
            print(f"  ➤ Grouped | DB: {db} | Table: {tbl} | Template: {tmpl} | Row index: {idx} | Column: {col}")
        else:
            _, db, tbl, u_key, col = first_missing_info
            print(f"  ➤ Ungrouped | DB: {db} | Table: {tbl} | Entry: {u_key} | Column: {col}")

    return count_with_keywords, count_missing_keywords

present, missing = count_keywords_status(file)
print(f"✅ Columns with keywords: {present}")
print(f"❌ Columns missing keywords: {missing}")
print(f"📊 Total columns: {present + missing}")


✅ Columns with keywords: 134095
❌ Columns missing keywords: 0
📊 Total columns: 134095
